In [1]:
import os
root = "/root/code_Bao/Vietnamese_stocks_forecasting" # ubuntu
# root = 'D:\Project\Vietnamese_stocks_forecasting' # win
os.chdir(root)

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Draw
import math

from vnstock import * #Load vietnamese data
import holidays # Load Vietnamese holidays

from tqdm import tqdm
import yaml

from src.utils.config_parse import DataConfigReader

# Read data

In [3]:
symbol = 'MWG'
start_date = "2021-01-01" #(YYYY-MM-DD)
end_date = '2023-07-18'

In [4]:
data_config = DataConfigReader(
    data_config_path='config/raw_data_config.yaml',
    dtype_config_path='config/data_type_dict.yaml',
    section='Historical_price'
)

In [6]:
data_path = 'data/raw/historical_price/MWG.csv'
df = pd.read_csv(data_path, usecols= data_config.read_column_names(), 
                 dtype=data_config.read_data_type())
df.head()

,time,open,high,low,close,volume,ticker
0,2021-01-04,39080,39430,39010,39430,1147400,MWG
1,2021-01-05,39230,40090,39100,39860,967500,MWG
2,2021-01-06,40210,40210,39460,39660,995500,MWG
3,2021-01-07,39400,39630,39130,39500,1118100,MWG
4,2021-01-08,39500,40180,39500,39830,1277600,MWG


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   time    132 non-null    object  
 1   open    132 non-null    int64   
 2   high    132 non-null    int64   
 3   low     132 non-null    int64   
 4   close   132 non-null    int64   
 5   volume  132 non-null    int64   
 6   ticker  132 non-null    category
dtypes: category(1), int64(5), object(1)
memory usage: 6.4+ KB


In [8]:
df.columns

Index(['time', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='object')